In [1]:
## we uploaded all the village files that I generated in 
## to google collab by going to the Files section on the left side of this page right clicking creating a new folder 'villages'
## and pressing the upload button to upload all the files from your local computer
from os import listdir
district_files = listdir("districts")
# Village files contains all the villages seperated by districs




In [2]:
## we want to make queries to the api by district 
## that way we can keep track of all the api calls that have been made and 
## restart from where we left off if an api call fails
import csv
import requests
all_villagecodes = []
for district in district_files: ## iterate through the district files 

    with open(f'districts/{district}', 'r') as villages_csv: # open each district file
        ## read the CSV file and turn each line into a dictionary where the csv header is the key to the dictionary
        villages = csv.DictReader(villages_csv)

        for village in villages:
            all_villagecodes.append(village) ## add villagecode the list we instantiated
        ## after this for loop is finished the all_villagecodes list will contain 
        ## all the villagecodes for  all the districts


In [4]:

#let's chunk this list in to lists of 100 villagecodes so that we can use comma seperated api call
villagecode_chunks = [ all_villagecodes[i:i + 100] for i in range(0, len(all_villagecodes), 100)] 

In [14]:

all_responses = []
start_no = 0 #which chunk to start from
for chunk_no,chunk in enumerate(villagecode_chunks[start_no:]): 
    
    villagecodes =  [c['villagecode'] for c in chunk] ## get all villagecodes in the chunk
    village_dict = {}
    for row in chunk:
        village_dict[row['villagecode']] = row #create dictionary of the village code we can use to lookup the latitude, longitude, and name since it doesnt come back in the response
    res = requests.get(f'http://api.nightlights.io/months/2010.1-2013.12/villages/{",".join(villagecodes)}')
    if (res.status_code != 200):
        print("response failed at chunk #", chunk_no + start_no)
    else:
        chunk_json = res.json()
        for row in chunk_json: ##add name lat and log from the dictionary we built to the response of the api call
            row["name"] = village_dict[row['villagecode']]['name']
            row["latitude"] = village_dict[row['villagecode']]['latitude']
            row["longitude"] = village_dict[row['villagecode']]['longitude']
        
        with open(f'output/{chunk_no+start_no}', 'w') as output: ## write chunk to disk, its easier to resume the job if we write frequently
            writer = csv.DictWriter(output, fieldnames=chunk_json[0].keys())
            if chunk_no+start_no == 0:
                writer.writeheader()
            for row in chunk_json:
                writer.writerow(row)